# 权限和分组
- 登录、注销和登录限制：

## 登录
- 在使用authenticate验证之后，如果验证通过了，那么就会返回一个user对象，拿到user对象后，使用django.contrib.auth.login进行登录

## 注销
- 注销，或者说退出登录，可以通过django.contrib.auth.logout来实现，这会清除掉用户的session数据

## 登录限制
- 有时候，某个视图函数是需要经过登录后才能访问的，那么可以通过django.contrib.decorators.login_required装饰器来实现

In [ ]:
# 登录与注销登录限制，详细代码见django_abstractbaseuser
# 注意方法名不能写成login，因为django中内置的有这个方法
class LoginView(View):
    def get(self, request):
        return render(request, "login.html")

    def post(self, request):
        form = UserForm(request.POST)
        if form.is_valid():
            telephone = form.cleaned_data.get("telephone")
            password = form.cleaned_data.get("password")
            remember = form.cleaned_data.get("remember")
            print(remember)
            user = authenticate(request, username=telephone, password=password)
            if user and user.is_active:
                # 如果不使用django自带的login函数，就只能像下面这样保存
                # request.session['user_id'] = user.id
                # 使用login登录
                login(request, user)
                if remember:
                    print(remember)
                    # 设置为None，记住我的session
                    request.session.set_expiry(0)
                else:
                    # 设置为0，表示当浏览器退出就清除session
                    request.session.set_expiry(None)
                # 这就是当未登录的用户访问个人中心时，先让他登录，登录成功后跳转到个人中心页面
                next_url = request.GET.get("next")
                if next_url:
                    return redirect(reverse("profile"))
                else:
                    return HttpResponse("登录成功")
            else:
                return HttpResponse("手机号码或密码错误")
        else:
            errors = form.errors.get_json_data()
            print(errors)
            return redirect(reverse("login"))


# 使用logout注销登录
def my_logout(request):
    logout(request)
    return HttpResponse("成功退出登录")


# 登录限制
# 经过@login_required装饰以后，如果没有登录，是访问不到该页面的，会报错
# 正常来说，如果未登录直接访问该页面应该让浏览器跳转到登录页面，所以给@login_required加参数
# 让他变成@login_required(login_url="/login/")。直到用户登录以后，就能访问个人中心页面了
# 但是要注意，用户登录以后，需要自己设置它跳转的页面，见上面的next_url
@login_required(login_url="/login/")
def profile(request):
    return HttpResponse("这是你的个人中心，只有登录以后才能看得到")

# 权限
- 导入：from django.contrib.auth.models import Permission, ContentType
- django中内置了权限的功能，它的权限都是针对表或者模型级别的。比如归某个模型上的数据是否可以进行增删改查操作。它不能针对数据级别的，比如对某个表中的某条数据进行增删改查操作（如果要实现数据级别的，考虑使用django_guardian）。创建完一个模型之后，针对这个模型权限默认就有四种种，分别是增，删，改，查，可以在执行完migrate命令后，查看数据库中的auth_permission表中的所有权限
- 在表中有一个表叫做：auth_permission记录着所有模型的相关权限，其中codename是表示这个这个权限的的名字，name是用来描述该权限的具体作用，content_type是用来表示该模型该权限是属于是哪个APP的下哪个模型的

## 通过定义模型添加权限：
- 如果我们想要增加新的权限，比如查看某个模型的权限，那么我们可以在定义模型的时候在Meta中定义属性permissions = ((code_name, name),())
        class Magazine(models.Model):
            title = models.CharField(max_length=100)
            author = models.ForeignKey(User, on_delete=models.CASCADE)

            class Meta:
                permissions = (
                    ("view_magazines", "查看杂志"),
                )
                

## 通过代码添加权限
- 权限都是django.contrib.auth.Permission的实例，这个模型包含一个字段，name，codename已经content_type，
        def add_permissions(request):
            content_type = ContentType.objects.get_for_model(Magazine)  # get_for_models批量添加权限
            permission = Permission.objects.create(codename="black_magazines", name="把垃圾文章拉黑", content_type=content_type)
            return HttpResponse("权限创建成功")

# 用户与权限管理
- 权限本身只是一个数据，必须和用户进行绑定，才能起到作用，User模型和权限之间的管理，可以通过以下几种方式管理：
    - 1.myuser.user_permissions.set(permission_list):直接给定一个权限的列表
    - 2.myuser.user_permissions.add(permission, permission,...):一个个添加权限
    - 3.myuser.user_permissions.remove(permission, permission,.):一个个删除权限
    - 4.myuser.user_permissions.clear():清除权限
    - 5.myuser.has_perm('app_name.codename'):判断是否拥有某个权限，权限参数是一个字符串，格式时app_name.codename。
    - 6.myuser.get_all_permissions():获取所有的权限

## 权限限定装饰器
- 使用django.contrib.auth.decorators.permission_required可以非常方便的检查用户是否拥有这个权限，如果拥有，那么就可以进入到指定的视图函数中，如果不拥有，那么就会报404错误
- 使用：@permission_required([权限列表],login_url, raise_exception)：第二、三参数为非必要参数
    - 参数：第一个表示访问这个页面你需要什么权限，第二个参数表示若没有登录你将跳转到登录页面登录,第三个参数表示若没有这个权限你将得到403页面
        @permission_required("front.change_magazine", login_url="/login/", raise_exception=True)
        def add_magazines(request):
            # 先判断用户是否登录：
            # if request.user.is_authenticated:
            #     if request.user.has_perm("front.change_magazine"):
            #         return HttpResponse("这是增加文章杂志的页面")
            #     else:
            #         return HttpResponse("你没有权限访问该页面", status=403)
            # else:
            #     return redirect(reverse("login"))

            # django内置了一个装饰器，能起到上述代码的作用
            return HttpResponse("这是增加文章的页面")
            
- 详细代码见django_abstractbaseuser
